# Install, import and initialize
First, we shall import basic libraries for handling dataset and training the model

In [2]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.datasets import fashion_mnist

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

# Import W&B and login
Login the Weights & bias service with the given token

In [3]:
import wandb
from wandb.keras import WandbCallback

project_name = 'project-fashion-mnist'
# %env WANDB_API_KEY='1af04e33e3d441eb82eb612e2c001eddec29bccb'

wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Avni Eshchar/.netrc


True

# Download and prepare the dataset
We download the dataset and normalize it to the 0-1 range

In [4]:
from class_names import class_names

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Subsetting train data and normalizing to [0., 1.]
x_train, x_test = x_train / 255., x_test / 255.
input_shape = (28, 28, 1)
num_classes = len(class_names)

print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

Shape of x_train:  (60000, 28, 28)
Shape of y_train:  (60000,)
Shape of x_test:  (10000, 28, 28)
Shape of y_test:  (10000,)


# Define a model

Here, we define a standard CNN (with convolution and max-pooling) in Keras.

In [5]:
def Model():
    inputs = keras.layers.Input(shape=input_shape)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(inputs)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.Conv2D(filters=28, kernel_size=(3, 3), activation='relu')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)

    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dense(28, activation='relu')(x)

    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)

    return keras.models.Model(inputs=inputs, outputs=outputs, )

# Train the model

In [6]:
# Initialize wandb with your project name
run = wandb.init(project=project_name,
                 config={
                     "learning_rate": 0.001,
                     "epochs": 5,
                     "batch_size": 32,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN",
                     "dataset": "fashion_mnist"
                 })
config = wandb.config  # We'll use this to configure our experiment

# Initialize model like you usually do.
keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do.
optimizer = tf.keras.optimizers.Adam(config.learning_rate) 
model.compile(optimizer, config.loss_function, metrics=['acc'])

wandb: Currently logged in as: eshcharc. Use `wandb login --relogin` to force relogin


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 28)        280       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 28)        7084      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 28)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 28)        7084      
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 28)          7084      
                                                             

In [7]:
# We focus on a subset of images, since this is for human review
val_images, val_labels = x_test[:32], y_test[:32]

_ = model.fit(x_train, y_train,
              epochs=config.epochs, 
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image', 
                                       validation_data=(val_images, val_labels), 
                                       labels=class_names)])

run.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/5
2/2 [==============================] - 0s 3ms/step0s - loss: 0.8110 - acc: 0.68


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230111_065736-1l20wb5l\files\model-best\assets


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230111_065736-1l20wb5l\files\model-best\assets
wandb: Adding directory to artifact (d:\dev\elbit\ML-course\project-2\wandb\run-20230111_065736-1l20wb5l\files\model-best)... Done. 0.0s


1875/1875 [==============================] - 47s 24ms/step - loss: 0.8109 - acc: 0.6874 - val_loss: 0.6455 - val_acc: 0.7563
Epoch 2/5
2/2 [==============================] - 0s 3ms/step0s - loss: 0.5339 - acc: 0.79


...

1875/1875 [==============================] - 44s 24ms/step - loss: 0.3750 - acc: 0.8620 - val_loss: 0.3973 - val_acc: 0.8557
Epoch 5/5
2/2 [==============================] - 0s 2ms/step0s - loss: 0.3353 - acc: 0.87


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230111_065736-1l20wb5l\files\model-best\assets


INFO:tensorflow:Assets written to: d:\dev\elbit\ML-course\project-2\wandb\run-20230111_065736-1l20wb5l\files\model-best\assets
wandb: Adding directory to artifact (d:\dev\elbit\ML-course\project-2\wandb\run-20230111_065736-1l20wb5l\files\model-best)... Done. 0.0s


1875/1875 [==============================] - 50s 27ms/step - loss: 0.3354 - acc: 0.8781 - val_loss: 0.3428 - val_acc: 0.8790


acc,▁▅▇▇█
epoch,▁▃▅▆█
loss,█▄▂▂▁
val_acc,▁▅▅▇█
val_loss,█▄▄▂▁
acc,0.87812
best_epoch,4
best_val_loss,0.34277
epoch,4
loss,0.33542
val_acc,0.879


## Hyperparameters search
At start, there was an attempt to use W&B swip for hyper search, but it failed every time, probably of configurations issues.

So, I investigated for most recommended tuners, and came up with keras-tuner as one of the them.

Model is declared in a model_builder function among with hyper parameters to tweek on.

In [15]:
import keras_tuner as kt
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.losses import SparseCategoricalCrossentropy

def model_builder(hp):
  hp_conv_units = hp.Int('hp_conv_units', min_value=32, max_value=64, step=4)
  hp_kernel = hp.Int('hp_kernel', min_value=3, max_value=6, step=1)
  hp_strides = hp.Int('hp_strides', min_value=1, max_value=4)
  hp_pool_size = hp.Int('hp_pool_size', min_value=2, max_value=5)
  hp_dense_units = hp.Int('hp_dense_units', min_value=50, max_value=200, step=10)
  hp_learning_rate = hp.Float('learning_rate', min_value=0.0005, max_value=0.01, sampling="log")
  ht_activation = hp.Choice("activation", ["relu", "tanh"])

  model = keras.Sequential([
    Input(shape=input_shape),

    Conv2D(filters=hp_conv_units, kernel_size=hp_kernel, strides=(hp_strides, hp_strides), activation=ht_activation, padding="SAME"),
    MaxPooling2D(pool_size=hp_pool_size, padding="SAME"),

    Conv2D(filters=hp_conv_units*2, kernel_size=hp_kernel*2, strides=(hp_strides, hp_strides), activation=ht_activation, padding="SAME"),
    MaxPooling2D(pool_size=hp_pool_size, padding="SAME"),

    Conv2D(filters=hp_conv_units*4, kernel_size=hp_kernel*4, strides=(hp_strides, hp_strides), activation=ht_activation, padding="SAME"),
    MaxPooling2D(pool_size=hp_pool_size, padding="SAME"),

    Flatten(),
    Dense(units=hp_dense_units, activation='relu'),
    Dense(units=num_classes, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=False,
      ignore_class=None,
      name='sparse_categorical_crossentropy'
    ),
    metrics=['accuracy'])
  
  return model

model = model_builder(kt.HyperParameters())
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 128)         1179776   
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 4, 4, 128)       

## Perform a Hyper parameters search
In the beginning, I used the whole dataset for hyper searching params. That took quite a long time (about 30+ hours for 1 cycle).

I realized that in ML, it is always a question of available time and computer resources, which (in my case) I had neither.

Actually, in hyper search, I only need to reach the best parameters' set, not to have the most trained model (which should come later).

Working with only a small equaly-distributed portion of the dataset in which the data was able to converge, can lead to the same results faster.

In [126]:
frac = 0.1
df = pd.DataFrame({ 'target': y_train })
df = df.groupby('target').apply(lambda x: x.sample(frac=frac))
df = df.droplevel('target')
index = df.index
X_sampled = x_train[index]
y_sampled = y_train[index]
print(f'Number of samples for Hyper-search: {len(x_train)}')

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [127]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

tuner = kt.Hyperband(model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='hp_search',
    overwrite=False,
    project_name='predict_fashion_item')

es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)
mc = ModelCheckpoint('./fashion_mnist.hd5', monitor='val_accuracy', verbose=1)

tuner.search(X_sampled, y_sampled, epochs=10, validation_split=0.2, callbacks=[es, mc])

# Get the optimal model with best params
model = tuner.get_best_models()[0]

INFO:tensorflow:Reloading Oracle from existing project hp_search\predict_fashion_item\oracle.json


INFO:tensorflow:Reloading Oracle from existing project hp_search\predict_fashion_item\oracle.json


INFO:tensorflow:Reloading Tuner from hp_search\predict_fashion_item\tuner0.json


INFO:tensorflow:Reloading Tuner from hp_search\predict_fashion_item\tuner0.json



Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
60                |48                |hp_conv_units
6                 |5                 |hp_kernel
2                 |3                 |hp_strides
3                 |3                 |hp_pool_size
110               |90                |hp_dense_units
0.0013908         |0.00052171        |learning_rate
relu              |relu              |activation
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
150/150 [==============================] - ETA: 0s - loss: 1.1972 - accuracy: 0.5058
Epoch 1: saving model to .\fashion_mnist.hd5


INFO:tensorflow:Assets written to: .\fashion_mnist.hd5\assets


INFO:tensorflow:Assets written to: .\fashion_mnist.hd5\assets


150/150 [==============================] - 181s 1s/step - loss: 1.1972 - accuracy: 0.5058 - val_loss: 13.5660 - val_accuracy: 0.0000e+00
Epoch 2/2
 96/150 [==================>...........] - ETA: 1:03 - loss: 0.6864 - accuracy: 0.7122

# Fit model based on best params

In [12]:
# model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[es])

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
 339/1500 [=====>........................] - ETA: 11:47 - loss: 0.2844 - accuracy: 0.9027

KeyboardInterrupt: 

# Evaluate model

In [8]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
79/79 [==============================] - 5s 64ms/step - loss: 0.3160 - accuracy: 0.9086
test loss, test acc: [0.3160328269004822, 0.9085999727249146]


# Save Trained Best Model

In [9]:
model.save('./fashion_mnist.hd5')

INFO:tensorflow:Assets written to: ./fashion_mnist.hd5\assets


INFO:tensorflow:Assets written to: ./fashion_mnist.hd5\assets
